In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import tensorflow as tf
import time

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

In [2]:
# load data: digits 5 to 9, but still label with 0 to 4, 
# because TensorFlow expects label's integers from 0 to n_classes-1.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

X_train2_full = mnist.train.images[mnist.train.labels >= 5]
y_train2_full = mnist.train.labels[mnist.train.labels >= 5] - 5
X_valid2_full = mnist.validation.images[mnist.validation.labels >= 5]
y_valid2_full = mnist.validation.labels[mnist.validation.labels >= 5] - 5
X_test2 = mnist.test.images[mnist.test.labels >= 5]
y_test2 = mnist.test.labels[mnist.test.labels >= 5] - 5

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# we want to keep only 100 instances per class in the training set 
# and let's keep only 30 instances per class in the validation set
# tesing set is already loaded above
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

## 3.1 Train Softmax Only

In [5]:
reset_graph() # clean the graph

no_update_tolerant = 20 # set the criteria that if the model didn't get better for how many epochs

no_update_progress = 0 # the variables to record for the not better epochs
current_best_accuracy = 0.0 # variables to record the best valid accuracy
current_best_loss = 10.0 # variables to record the best valid loss

saver = tf.train.Saver() # to store the model
with tf.Session() as sess: # start the tensorflow nn session
    restore_saver = tf.train.import_meta_graph("./Models/yen/mnist_5.ckpt.meta") # load meta graph
    restore_saver.restore(sess, tf.train.latest_checkpoint('./Models/yen/')) # restore weights

    """
    Graph Define Step
    
    1. Restore the variable about the component in graph which help to retrieve and assign value
    """
    graph = tf.get_default_graph() # get original graph
    X = graph.get_tensor_by_name("Input_X:0") # assign placeholder variables with input X
    y = graph.get_tensor_by_name("Input_Label:0")# assign placeholder variables with input Label
    loss = graph.get_tensor_by_name("Loss:0") # assign variables about loss
    Y_proba = graph.get_tensor_by_name("Softmax_Layer/Softmax:0") # assign variables about loss
    accuracy = graph.get_tensor_by_name("Accuracy:0") # assign variables  about accuracy

    """
    2. Get the softmax layer which is what we would like to train
    """
    output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Softmax_Layer") # get softmax layer by scope name
    
    """
    3. Define the optimizer and assign to only update on softmax_layer which defined above
    """
    optimizer = tf.train.AdamOptimizer(0.0017, name="Adam_Op") # create a new optimizer
    training_op = optimizer.minimize(loss, var_list=output_layer_vars) # minimize the loss and only update certern variables
    var_list_init = tf.variables_initializer([optimizer.get_slot(var, name) for name in optimizer.get_slot_names() for var in output_layer_vars]+list(optimizer._get_beta_accumulators())) # to get the variables name about the softmax layer and 
    sess.run(var_list_init) # initialize the un-initizlized variables
    """
    Training Step
    """
    begin_time = time.time() # record the begin training time
    for epochs in range(1000): # training on 1000 epochs
        _, train_loss = sess.run([training_op, loss], feed_dict={X:X_train2, y:y_train2}) # feed with training variables
        valid_loss, valid_accuracy = sess.run([loss, accuracy], feed_dict={X:X_valid2, y:y_valid2}) # feed with validation variables
        test_loss ,  test_accuracy = sess.run([loss, accuracy], feed_dict={X:X_test2, y:y_test2}) # feed with testing variables
        
        if valid_accuracy > current_best_accuracy: # if better
            current_best_accuracy = valid_accuracy # keep record
            save_path = saver.save(sess, "./Models/TransferLearning/HW3_1/Team26_HW3_1.ckpt") #keep the best model
            no_update_progress = 0
        else:                                      # if not better
            no_update_progress += 1                # record on no update

        if valid_loss < current_best_loss: # if better
            current_best_loss = valid_loss # keep record       
            
        if no_update_progress == no_update_tolerant: break # stop training
            
                
        print("{:4d} epochs,   Validation loss:{:.5f},   Best loss:{:.5f},   Accuracy:{:.2f}%/ {:.2f}%".format(
            epochs+1,
            valid_loss,
            current_best_loss,                                                                                       
            valid_accuracy*100,
            test_accuracy*100
        )) # print the performane for each epochs
    finish_itme = time.time() # record the finish time
    print("\nTraining Time:{:3f}".format(finish_itme-begin_time)) # calculate and print training time
        
        

INFO:tensorflow:Restoring parameters from ./Models/yen/mnist_5.ckpt
   1 epochs,   Validation loss:1.52103,   Best loss:1.52103,   Accuracy:36.67%/ 41.56%
   2 epochs,   Validation loss:1.51702,   Best loss:1.51702,   Accuracy:37.33%/ 42.05%
   3 epochs,   Validation loss:1.51312,   Best loss:1.51312,   Accuracy:38.00%/ 42.42%
   4 epochs,   Validation loss:1.50918,   Best loss:1.50918,   Accuracy:39.33%/ 42.97%
   5 epochs,   Validation loss:1.50506,   Best loss:1.50506,   Accuracy:39.33%/ 43.16%
   6 epochs,   Validation loss:1.50065,   Best loss:1.50065,   Accuracy:38.67%/ 43.43%
   7 epochs,   Validation loss:1.49589,   Best loss:1.49589,   Accuracy:39.33%/ 43.65%
   8 epochs,   Validation loss:1.49083,   Best loss:1.49083,   Accuracy:40.00%/ 43.92%
   9 epochs,   Validation loss:1.48557,   Best loss:1.48557,   Accuracy:40.67%/ 44.15%
  10 epochs,   Validation loss:1.48024,   Best loss:1.48024,   Accuracy:41.33%/ 44.44%
  11 epochs,   Validation loss:1.47491,   Best loss:1.47491,  

 112 epochs,   Validation loss:1.17693,   Best loss:1.17693,   Accuracy:74.67%/ 71.10%
 113 epochs,   Validation loss:1.17589,   Best loss:1.17589,   Accuracy:74.67%/ 71.28%
 114 epochs,   Validation loss:1.17494,   Best loss:1.17494,   Accuracy:74.67%/ 71.30%
 115 epochs,   Validation loss:1.17407,   Best loss:1.17407,   Accuracy:74.67%/ 71.47%
 116 epochs,   Validation loss:1.17328,   Best loss:1.17328,   Accuracy:74.67%/ 71.61%
 117 epochs,   Validation loss:1.17256,   Best loss:1.17256,   Accuracy:74.67%/ 71.67%
 118 epochs,   Validation loss:1.17189,   Best loss:1.17189,   Accuracy:74.67%/ 71.86%
 119 epochs,   Validation loss:1.17126,   Best loss:1.17126,   Accuracy:74.67%/ 71.92%
 120 epochs,   Validation loss:1.17064,   Best loss:1.17064,   Accuracy:74.00%/ 71.94%
 121 epochs,   Validation loss:1.17002,   Best loss:1.17002,   Accuracy:74.00%/ 72.00%
 122 epochs,   Validation loss:1.16937,   Best loss:1.16937,   Accuracy:74.00%/ 72.17%
 123 epochs,   Validation loss:1.16865,   B

## 3.2 Cache the Output of FIfth layer for Speed Up

In [6]:
reset_graph() # clean the graph

no_update_tolerant = 20 # set the criteria that if the model didn't get better for how many epochs

no_update_progress = 0 # the variables to record for the not better epochs
current_best_accuracy = 0.0 # variables to record the best valid accuracy
current_best_loss = 10.0 # variables to record the best valid loss

saver = tf.train.Saver() # to store the model
with tf.Session() as sess: # start the tensorflow nn session
    restore_saver = tf.train.import_meta_graph("./Models/yen/mnist_5.ckpt.meta") # load meta graph
    restore_saver.restore(sess, tf.train.latest_checkpoint('./Models/yen/')) # restore weights
#     sess.run(tf.global_variables_initializer()) # initialize the weights 
#     sess.run(tf.local_variables_initializer()) # initialize the local variables hidden in the tf.metrics.recallmethod.

    """
    Graph Define Step
    
    1. Restore the variable about the component in graph which help to retrieve and assign value
    """
    graph = tf.get_default_graph() # get original graph
    X = graph.get_tensor_by_name("Input_X:0") # assign placeholder variables with input X
    y = graph.get_tensor_by_name("Input_Label:0")# assign placeholder variables with input Label
    loss = graph.get_tensor_by_name("Loss:0") # assign variables about loss
    Y_proba = graph.get_tensor_by_name("Softmax_Layer/Softmax:0") # assign variables about loss
    accuracy = graph.get_tensor_by_name("Accuracy:0") # assign variables  about accuracy
    
    fifth_layer_out = graph.get_tensor_by_name("Fully_Connected_Layer_5/Elu:0").op.outputs[0] # get the output tensors of fifth's layers

    
    """
    2. Get the softmax layer which is what we would like to train
    """
    output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Softmax_Layer") # get softmax layer by scope name
    
    """
    3. Define the optimizer and assign to only update on softmax_layer which defined above
    """
    optimizer = tf.train.AdamOptimizer(0.0017, name="Adam_Op")
    training_op = optimizer.minimize(loss, var_list=output_layer_vars) # minimize the loss and only update certern variables
    var_list_init = tf.variables_initializer([optimizer.get_slot(var, name) for name in optimizer.get_slot_names() for var in output_layer_vars]+list(optimizer._get_beta_accumulators()))# to get the variables name about the softmax layer and 

    sess.run(var_list_init) # initialize the un-initialized value
    
    """
    Get the training output of fifth layer
    """
    fifth_layer_out_cache = sess.run(fifth_layer_out, feed_dict={X:X_train2, y:y_train2}) # feed in the training value and get the fifth layer value out
    
    """
    Training Step
    """
    begin_time = time.time()
    for epochs in range(1000): # training on 1000 epochs
        _, train_loss = sess.run([training_op, loss], feed_dict={fifth_layer_out:fifth_layer_out_cache, y:y_train2}) # feed with training variables
        valid_loss, valid_accuracy = sess.run([loss, accuracy], feed_dict={X:X_valid2, y:y_valid2}) # feed with validation variables
        test_loss ,  test_accuracy = sess.run([loss, accuracy], feed_dict={X:X_test2, y:y_test2}) # feed with testing variables
        
        if valid_accuracy > current_best_accuracy: # if better
            current_best_accuracy = valid_accuracy # keep record
            save_path = saver.save(sess, "./Models/TransferLearning/HW3_2/Team26_HW3_2.ckpt") #keep the best model
            no_update_progress = 0
        else:                                      # if not better
            no_update_progress += 1                # record on no update

        if valid_loss < current_best_loss: # if better
            current_best_loss = valid_loss # keep record            
            
        if no_update_progress == no_update_tolerant: break # stop training
            
                
        print("{:4d} epochs,   Validation loss:{:.5f},   Best loss:{:.5f},   Accuracy:{:.2f}%/ {:.2f}%".format(
            epochs+1,
            valid_loss,
            current_best_loss,                                                                                       
            valid_accuracy*100, # to show as % we need to *100
            test_accuracy*100 # to show as % we need to *100
        )) # print the performane for each epochs
    finish_itme = time.time() # record the finish time
    print("\nTraining Time:{:3f}".format(finish_itme-begin_time)) # calculate and print training time
        
        

INFO:tensorflow:Restoring parameters from ./Models/yen/mnist_5.ckpt
(500, 128)
   1 epochs,   Validation loss:1.52103,   Best loss:1.52103,   Accuracy:36.67%/ 41.56%
   2 epochs,   Validation loss:1.51702,   Best loss:1.51702,   Accuracy:37.33%/ 42.05%
   3 epochs,   Validation loss:1.51312,   Best loss:1.51312,   Accuracy:38.00%/ 42.42%
   4 epochs,   Validation loss:1.50918,   Best loss:1.50918,   Accuracy:39.33%/ 42.97%
   5 epochs,   Validation loss:1.50506,   Best loss:1.50506,   Accuracy:39.33%/ 43.16%
   6 epochs,   Validation loss:1.50065,   Best loss:1.50065,   Accuracy:38.67%/ 43.43%
   7 epochs,   Validation loss:1.49589,   Best loss:1.49589,   Accuracy:39.33%/ 43.65%
   8 epochs,   Validation loss:1.49083,   Best loss:1.49083,   Accuracy:40.00%/ 43.92%
   9 epochs,   Validation loss:1.48557,   Best loss:1.48557,   Accuracy:40.67%/ 44.15%
  10 epochs,   Validation loss:1.48024,   Best loss:1.48024,   Accuracy:41.33%/ 44.44%
  11 epochs,   Validation loss:1.47491,   Best loss

 121 epochs,   Validation loss:1.17002,   Best loss:1.17002,   Accuracy:74.00%/ 72.00%
 122 epochs,   Validation loss:1.16937,   Best loss:1.16937,   Accuracy:74.00%/ 72.17%
 123 epochs,   Validation loss:1.16865,   Best loss:1.16865,   Accuracy:74.00%/ 72.25%
 124 epochs,   Validation loss:1.16786,   Best loss:1.16786,   Accuracy:74.00%/ 72.37%
 125 epochs,   Validation loss:1.16698,   Best loss:1.16698,   Accuracy:74.00%/ 72.43%
 126 epochs,   Validation loss:1.16598,   Best loss:1.16598,   Accuracy:74.00%/ 72.58%
 127 epochs,   Validation loss:1.16488,   Best loss:1.16488,   Accuracy:74.00%/ 72.64%
 128 epochs,   Validation loss:1.16367,   Best loss:1.16367,   Accuracy:74.00%/ 72.78%
 129 epochs,   Validation loss:1.16236,   Best loss:1.16236,   Accuracy:74.67%/ 72.99%

Training Time:0.658595


## 3.3 Train on 5th Layer + Softmax

In [17]:
reset_graph() # clean the graph

no_update_tolerant = 20 # set the criteria that if the model didn't get better for how many epochs

no_update_progress = 0 # the variables to record for the not better epochs
current_best_accuracy = 0.0 # variables to record the best valid accuracy
current_best_loss = 10.0 # variables to record the best valid loss

saver = tf.train.Saver() # to store the model
with tf.Session() as sess: # start the tensorflow nn session
    restore_saver = tf.train.import_meta_graph("./Models/yen/mnist_5.ckpt.meta") # load meta graph
    restore_saver.restore(sess, tf.train.latest_checkpoint('./Models/yen/')) # restore weights
#     sess.run(tf.global_variables_initializer()) # initialize the weights 
#     sess.run(tf.local_variables_initializer()) # initialize the local variables hidden in the tf.metrics.recallmethod.

    """
    Graph Define Step
    
    1. Restore the variable about the component in graph which help to retrieve and assign value
    """
    graph = tf.get_default_graph() # get original graph
    X = graph.get_tensor_by_name("Input_X:0") # assign placeholder variables with input X
    y = graph.get_tensor_by_name("Input_Label:0")# assign placeholder variables with input Label
    
    fourth_layer_out = graph.get_tensor_by_name("Fully_Connected_Layer_4/Elu:0").op.outputs[0]
#     print(fourth_layer_out)
    
    """
    2. Get the training output of fourth layer
    """
    fourth_layer_out_cache = sess.run(fourth_layer_out, feed_dict={X:X_train2, y:y_train2}) # get the output of fourth layer
#     print(fourth_layer_out_cache.shape)
    temp = set(tf.global_variables())
    """
    3.Create new softmax layer and connect old fourth layer, connect to softmax layer
    """
    y_ = tf.layers.dense(inputs=fourth_layer_out, 
                             units=5, 
                             activation=tf.nn.softmax, #softmax
                             kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),
                             name = "New_Softmax_Layer"
                            )  # Layer using dense

    """
    4. Get the softmax layer which is what we would like to train
    """
    output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="New_Softmax_Layer") # get softmax layer by scope name

    
    """
    5. Define loss, optimizer and assign to only update on softmax_layer which defined above
    """   
    
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_, name="New_Cross_Entropy"), name="New_Loss") # Refine Loss Function

    optimizer = tf.train.AdamOptimizer(0.0017, name="Adam_Op")
    training_op = optimizer.minimize(loss, var_list=output_layer_vars) # minimize the loss and only update certern variables
    
    """
    6. Define new evaluation metrics
    """
    # Define Accuracy
    predicted_class = tf.argmax(y_, 1, output_type=tf.int32)
    correct_predict = tf.equal(y, predicted_class) # [True, False ..., True]
    accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32)) # [True, False ..., True] --> [1,0,...,1]

    
    """
    7. Initialize all the variables
    """
    var_list_init = tf.variables_initializer(set(tf.global_variables())-temp)
    sess.run(var_list_init) # init the variable about fourth + softmax layer + adam optimizer
   
    
    """
    Training Step
    """
    begin_time = time.time()
    for epochs in range(1000): # training on 1000 epochs
        _, train_loss = sess.run([training_op, loss], feed_dict={fourth_layer_out:fourth_layer_out_cache, y:y_train2}) # feed with training variables
        valid_loss, valid_accuracy = sess.run([loss, accuracy], feed_dict={X:X_valid2, y:y_valid2}) # feed with validation variables
        test_loss ,  test_accuracy = sess.run([loss, accuracy], feed_dict={X:X_test2, y:y_test2}) # feed with testing variables
        
        if valid_accuracy > current_best_accuracy: # if better
            current_best_accuracy = valid_accuracy # keep record
            save_path = saver.save(sess, "./Models/TransferLearning/HW3_3/Team26_HW3_3.ckpt") #keep the best model
            no_update_progress = 0
        else:                                      # if not better
            no_update_progress += 1                # record on no update

        if valid_loss < current_best_loss: # if better
            current_best_loss = valid_loss # keep record            
            
        if no_update_progress == no_update_tolerant: break # stop training
            
                
        print("{:4d} epochs,   Validation loss:{:.5f},   Best loss:{:.5f},   Accuracy:{:.2f}%/ {:.2f}%".format(
            epochs+1,
            valid_loss,
            current_best_loss,                                                                                       
            valid_accuracy*100,
            test_accuracy*100
        )) # print the performane for each epochs
    finish_itme = time.time() # record the finish time
    print("\nTraining Time:{:3f}".format(finish_itme-begin_time)) # calculate and print training time
        
        

INFO:tensorflow:Restoring parameters from ./Models/yen/mnist_5.ckpt
   1 epochs,   Validation loss:1.65729,   Best loss:1.65729,   Accuracy:20.00%/ 25.61%
   2 epochs,   Validation loss:1.64555,   Best loss:1.64555,   Accuracy:22.67%/ 26.85%
   3 epochs,   Validation loss:1.63323,   Best loss:1.63323,   Accuracy:23.33%/ 28.22%
   4 epochs,   Validation loss:1.62069,   Best loss:1.62069,   Accuracy:25.33%/ 29.62%
   5 epochs,   Validation loss:1.60814,   Best loss:1.60814,   Accuracy:25.33%/ 30.98%
   6 epochs,   Validation loss:1.59560,   Best loss:1.59560,   Accuracy:26.00%/ 32.24%
   7 epochs,   Validation loss:1.58288,   Best loss:1.58288,   Accuracy:26.00%/ 33.70%
   8 epochs,   Validation loss:1.56975,   Best loss:1.56975,   Accuracy:27.33%/ 35.10%
   9 epochs,   Validation loss:1.55610,   Best loss:1.55610,   Accuracy:28.67%/ 36.54%
  10 epochs,   Validation loss:1.54198,   Best loss:1.54198,   Accuracy:30.67%/ 37.83%
  11 epochs,   Validation loss:1.52762,   Best loss:1.52762,  

## 3.4 Only Frozen 3th & 4th Layers

In [22]:
reset_graph() # clean the graph

no_update_tolerant = 20 # set the criteria that if the model didn't get better for how many epochs

no_update_progress = 0 # the variables to record for the not better epochs
current_best_accuracy = 0.0 # variables to record the best valid accuracy
current_best_loss = 10.0 # variables to record the best valid loss

saver = tf.train.Saver() # to store the model
with tf.Session() as sess: # start the tensorflow nn session
    restore_saver = tf.train.import_meta_graph("./Models/yen/mnist_5.ckpt.meta") # load meta graph
    restore_saver.restore(sess, tf.train.latest_checkpoint('./Models/yen/')) # restore weights

    """
    Graph Define Step
    
    1. Restore the variable about the component in graph which help to retrieve and assign value
    """
    graph = tf.get_default_graph() # get original graph
    X = graph.get_tensor_by_name("Input_X:0") # assign placeholder variables with input X
    y = graph.get_tensor_by_name("Input_Label:0")# assign placeholder variables with input Label
    
    fourth_layer_out = graph.get_tensor_by_name("Fully_Connected_Layer_4/Elu:0").op.outputs[0]
    """
    2.Create new softmax layer and connect old fourth layer, connect to softmax layer
    """
    y_ = tf.layers.dense(inputs=fourth_layer_out, 
                             units=5, 
                             activation=tf.nn.softmax, #softmax
                             kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),
                             name = "New_Softmax_Layer"
                            )  # Layer using dense

    """
    3. Get the 1th, 2th and softmax layer which is what we would like to train
    """
    output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Fully_Connected_Layer_1") # get softmax layer by scope name
    output_layer_vars += tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Fully_Connected_Layer_2") # get softmax layer by scope name
    output_layer_vars += tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="New_Softmax_Layer") # get softmax layer by scope name

    
    """
    4. Define loss, optimizer and assign to only update on softmax_layer which defined above
    """   
    
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_, name="New_Cross_Entropy"), name="New_Loss") # Refine Loss Function

    optimizer = tf.train.AdamOptimizer(0.0017, name="Adam_Op")
    training_op = optimizer.minimize(loss) # minimize the loss and only update certern variables
    
    """
    5. Define new evaluation metrics
    """
    # Define Accuracy
    predicted_class = tf.argmax(y_, 1, output_type=tf.int32)
    correct_predict = tf.equal(y, predicted_class) # [True, False ..., True]
    accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32)) # [True, False ..., True] --> [1,0,...,1]

    
    """
    6. Initialize all the variables
    """
    var_list_init = tf.variables_initializer(set(tf.global_variables())-temp)
    sess.run(var_list_init) # init the variable about fourth + softmax layer + adam optimizer
   
    
    """
    Training Step
    """
    begin_time = time.time()
    for epochs in range(1000): # training on 1000 epochs
        _, train_loss = sess.run([training_op, loss], feed_dict={X:X_train2, y:y_train2}) # feed with training variables
        valid_loss, valid_accuracy = sess.run([loss, accuracy], feed_dict={X:X_valid2, y:y_valid2}) # feed with validation variables
        test_loss ,  test_accuracy = sess.run([loss, accuracy], feed_dict={X:X_test2, y:y_test2}) # feed with testing variables
        
        if valid_accuracy > current_best_accuracy: # if better
            current_best_accuracy = valid_accuracy # keep record
            save_path = saver.save(sess, "./Models/TransferLearning/HW3_4/Team26_HW3_4.ckpt") #keep the best model
            no_update_progress = 0
        else:                                      # if not better
            no_update_progress += 1                # record on no update

        if valid_loss < current_best_loss: # if better
            current_best_loss = valid_loss # keep record            
            
        if no_update_progress == no_update_tolerant: break # stop training
            
                
        print("{:4d} epochs,   Validation loss:{:.5f},   Best loss:{:.5f},   Accuracy:{:.2f}%/ {:.2f}%".format(
            epochs+1,
            valid_loss,
            current_best_loss,                                                                                       
            valid_accuracy*100,
            test_accuracy*100
        )) # print the performane for each epochs
    finish_itme = time.time() # record the finish time
    print("\nTraining Time:{:3f}".format(finish_itme-begin_time)) # calculate and print training time
        
        

INFO:tensorflow:Restoring parameters from ./Models/yen/mnist_5.ckpt
   1 epochs,   Validation loss:1.44409,   Best loss:1.44409,   Accuracy:45.33%/ 45.46%
   2 epochs,   Validation loss:1.34925,   Best loss:1.34925,   Accuracy:53.33%/ 51.92%
   3 epochs,   Validation loss:1.23956,   Best loss:1.23956,   Accuracy:71.33%/ 73.28%
   4 epochs,   Validation loss:1.19864,   Best loss:1.19864,   Accuracy:73.33%/ 76.36%
   5 epochs,   Validation loss:1.15297,   Best loss:1.15297,   Accuracy:76.00%/ 80.44%
   6 epochs,   Validation loss:1.10571,   Best loss:1.10571,   Accuracy:80.00%/ 84.65%
   7 epochs,   Validation loss:1.08129,   Best loss:1.08129,   Accuracy:84.00%/ 86.20%
   8 epochs,   Validation loss:1.06809,   Best loss:1.06809,   Accuracy:84.67%/ 87.08%
   9 epochs,   Validation loss:1.05245,   Best loss:1.05245,   Accuracy:86.00%/ 88.19%
  10 epochs,   Validation loss:1.04559,   Best loss:1.04559,   Accuracy:86.00%/ 88.91%
  11 epochs,   Validation loss:1.04551,   Best loss:1.04551,  